<a href="https://colab.research.google.com/github/Saharuth/MachineLearning/blob/main/classification_mnist.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sklearn.datasets import fetch_openml
mnist = fetch_openml('mnist_784', version=1)
mnist.keys()

In [ ]:
X, y = mnist['data'], mnist['target']
print(X.shape)
print(y.shape)

In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt

# print(X[0:1])
some_digit = X[0:1]
some_digit_arr = some_digit.to_numpy()
# print(some_digit_arr)
some_digit_image = some_digit_arr.reshape(28, 28)
# print(some_digit_image)
plt.imshow(some_digit_image, cmap = mpl.cm.binary, interpolation='nearest')
plt.axis('off')
plt.show()

print('Solution :' + y[0]) # string

In [ ]:
import numpy as np
y = y.astype(np.uint8) # cast y to integers

In [ ]:
X_train, X_test, y_train, y_test = X[:60000], X[60000:], y[:60000], y[60000:]

In [ ]:
print(X_train)

###Binary Classifier
######5-detector => capable of distinguishing between just two classes => 5, not 5

In [ ]:
y_train_5 = (y_train == 5)
y_test_5 = (y_test == 5)

In [8]:
from sklearn.linear_model import SGDClassifier

sgd_clf = SGDClassifier(random_state=42)
sgd_clf.fit(X_train, y_train_5)

SGDClassifier(random_state=42)

In [9]:
sgd_clf.predict(some_digit)

array([ True])

###Cross-Validation

In [10]:
# สร้างการ cross-validation ที่คล้ายกับ cross_val_score function
from sklearn.model_selection import StratifiedKFold
from sklearn.base import clone

skfolds = StratifiedKFold(n_splits=3, random_state=42, shuffle=True)
# n_splits=3 => ต้องการแบ่งข้อมูลเป็น 3 fold

for train_index, test_index in skfolds.split(X_train, y_train_5): # แบ่งข้อมูลออกเป็น fold
  clone_clf = clone(sgd_clf)
  # print(train_index)
  # print(test_index)
  X_train_folds = X_train.iloc[train_index]
  y_train_folds = y_train_5.iloc[train_index]
  X_test_fold = X_train.iloc[test_index]
  y_test_fold = y_train_5.iloc[test_index]

  clone_clf.fit(X_train_folds, y_train_folds)
  y_pred = clone_clf.predict(X_test_fold)
  n_correct = sum(y_pred == y_test_fold)
  print(n_correct/len(y_pred))

0.9669
0.91625
0.96785


In [11]:
from sklearn.model_selection import cross_val_score

cross_val_score(sgd_clf, X_train, y_train_5, cv=3, scoring="accuracy")

array([0.95035, 0.96035, 0.9604 ])

In [ ]:
# ประเมินความแม่นยำจากการเดาข้อมูลที่ไม่ใช่เลข 5
from sklearn.base import BaseEstimator

class Never5Classifier(BaseEstimator):
  def fit(self, X, y=None):
    pass
  def predict(self, X):
    return np.zeros((len(X), 1), dtype=bool)

never_5_clf = Never5Classifier()
cross_val_score(never_5_clf, X_train, y_train_5, cv=3, scoring='accuracy')

###Confusion Matrix
###### การนับจำนวนครั้งที่ของตัวอย่าง class A ที่ถูกทำนายเป็น class B

In [13]:
from sklearn.model_selection import cross_val_predict

y_train_pred = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3)
# return เป็น set ที่ predict ในแต่ละ test fold
# ซึ่ง model ไม่เคยเห็น data นี้มาก่อนในระหว่างการ train

###### row => actual class
###### column => predicted class

In [ ]:
from sklearn.metrics import confusion_matrix
confusion_matrix(y_train_5, y_train_pred)

######แถวที่ 1 ของ matrix พิจารณาเป็น non-5s images(negative class)
######   true nagative, false positive
######แถวที่ 2 ของ matrix พิจารณาเป็น 5s images (positive class)
######   false nagative, true positve

In [ ]:
y_train_perfect_predictions = y_train_5
confusion_matrix(y_train_5, y_train_perfect_predictions)

###Precision & Recall

In [16]:
from sklearn.metrics import precision_score, recall_score

print(precision_score(y_train_5, y_train_pred))
print(recall_score(y_train_5, y_train_pred))

0.8370879772350012
0.6511713705958311


### F1 Score
###### combine precision and recall
###### ถ้าต้องการทางง่ายๆที่เปรียบเทียบ 2 classifiers

In [17]:
from sklearn.metrics import f1_score

f1_score(y_train_5, y_train_pred)

0.7325171197343846

### precision/recall tradeoff

In [18]:
y_scores = sgd_clf.decision_function(some_digit)
print(y_scores)
threshold = 0
y_some_digit_pred = y_scores > threshold
print(y_some_digit_pred)

[2164.22030239]
[ True]


In [19]:
threshold = 8000
y_some_digit_pred = y_scores > threshold
print(y_some_digit_pred)

[False]


In [20]:
y_scores = cross_val_predict(sgd_clf, X_train, y_train_5, cv=3, method="decision_function")

In [ ]:
from sklearn.metrics import precision_recall_curve

precisions, recalls, thresholds = precision_recall_curve(y_train_5, y_scores)

In [ ]:
def plot_precision_recall_vs_threshold(precisions, recalls, thresholds):
  plt.plot(thresholds, precisions[:-1], "b--", label="Precision")
  plt.plot(thresholds, recalls[:-1], "g-", label="Recall")
  [...]

plot_precision_recall_vs_threshold(precisions, recalls, thresholds)
plt.show()

In [ ]:
threshold_90_precision = thresholds[np.argmax(precisions >= 0.90)]
print(threshold_90_precision)

In [ ]:
y_train_pred_90 = (y_scores >= threshold_90_precision)
print(y_train_pred_90)

In [ ]:
print(precision_score(y_train_5, y_train_pred_90))
print(recall_score(y_train_5, y_train_pred_90))

###ROC curve

In [25]:
from sklearn.metrics import roc_curve

fpr, tpr, thresholds = roc_curve(y_train_5, y_scores)